# Введение в MapReduce модель на Python


In [2]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [3]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [4]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [5]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [6]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [7]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [8]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [9]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [10]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [11]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [12]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [13]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [14]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [15]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [16]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.628311094757751),
 (1, 1.628311094757751),
 (2, 1.628311094757751),
 (3, 1.628311094757751),
 (4, 1.628311094757751)]

## Inverted index 

In [17]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [18]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [29]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [20]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('it', 18)]),
 (1, [('banana', 2), ('is', 18), ('what', 10)])]

## TeraSort

In [21]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.03806232055429637),
   (None, 0.05940208329562868),
   (None, 0.11624640477109727),
   (None, 0.13015640772234038),
   (None, 0.14185470847698323),
   (None, 0.1967156079952712),
   (None, 0.2205087488399471),
   (None, 0.28812568905772207),
   (None, 0.3023331064015665),
   (None, 0.3063699562350476),
   (None, 0.3065278806982673),
   (None, 0.31408859563690505),
   (None, 0.3443467669299216),
   (None, 0.44260614520523267),
   (None, 0.45118087282377506),
   (None, 0.4548372806610449)]),
 (1,
  [(None, 0.541428869132883),
   (None, 0.5733137200081921),
   (None, 0.5811272451749184),
   (None, 0.5916681806455112),
   (None, 0.6029271961413297),
   (None, 0.6093933687761275),
   (None, 0.6095343356990095),
   (None, 0.6507463357209563),
   (None, 0.7684912887130781),
   (None, 0.8363369377821143),
   (None, 0.9075279195203269),
   (None, 0.9481746768955),
   (None, 0.9658811613595257),
   (None, 0.9992273659372453)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [22]:
input = [1, 2, 3, 4, 5, 66, 7, 8, 9, 10]

def RECORDREADER():
  for (numid, num) in enumerate(input):
    yield ("{}".format(numid), num)

def MAP(numId:str, num:int):
  yield (0, num)
 
def REDUCE(numId:str, num:Iterator[int]):
	yield (numId, max(num))
    
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 66)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [23]:
input = [10, 10, 10, 10, 20, 20, 20, 20]

def RECORDREADER():
  for (numid, num) in enumerate(input):
    yield ("{}".format(numid), num)

def MAP(numId:str, num:int):
  yield (0, num)
 
def REDUCE(numId:str, num:Iterator[int]):
	num_list = list(num)
	yield (numId, sum(num_list)/len(num_list))
    
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 15.0)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [24]:
def groupbykey_sorted(iterable):
    sorted_iterable = sorted(iterable, key=lambda x: x[0])
    grouped = {}
    current_key = None
    current_values = []
    for key, value in sorted_iterable:
        if key is not current_key:
            if current_key is not None:
                grouped[current_key] = current_values
            current_key = key
            current_values = []
        current_values.append(value)
    grouped[current_key] = current_values
    return grouped.items()

example_1 = [('a', 1), ('b', 2), ('a', 3), ('b', 4), ('c', 5)]
example_2 = [('x', 10), ('y', 20), ('x', 30), ('z', 40), ('y', 50)]
example_3 = [('apple', 'red'), ('banana', 'yellow'), ('apple', 'green'), ('banana', 'green')]

grouped_1 = groupbykey_sorted(example_1)
grouped_2 = groupbykey_sorted(example_2)
grouped_3 = groupbykey_sorted(example_3)

grouped_1, grouped_2, grouped_3

(dict_items([('a', [1, 3]), ('b', [2, 4]), ('c', [5])]),
 dict_items([('x', [10, 30]), ('y', [20, 50]), ('z', [40])]),
 dict_items([('apple', ['red', 'green']), ('banana', ['yellow', 'green'])]))

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [25]:
input_values = [1, 1, 1, 1, 2, 3, 3, 2, 3]

maps = 3
reducers = 2

def RECORDREADER(split):
    for value in split:
        yield (value, None)
        
def INPUTFORMAT():
    global maps
    split_size = int(np.ceil(len(input_values) / maps))
    for i in range(0, len(input_values), split_size):
        yield RECORDREADER(input_values[i:i + split_size])

def MAP(value, _):
    yield (value, None)

def PARTITIONER(key, reducers=2):
    return hash(key) % reducers

def COMBINER(key, values):
    yield (key, None)

def REDUCE(key, values):
    yield (key, None)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

9 key-value pairs were sent over a network.


[(0, [(2, None)]), (1, [(1, None), (3, None)])]

# Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [26]:
input = [1, 3, 4, 5, 98, 1000, 0, 12, 15, 15, 15]

pred = lambda x: x % 2 == 0

def RECORDREADER():
  for (numid, num) in enumerate(input):
    yield ("{}".format(numid), num)

def MAP(numId:str, value):
	if pred(value):
		yield (value, value)

def REDUCE(key, values):
    yield (key, values)
    
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(4, [4]), (98, [98]), (1000, [1000]), (0, [0]), (12, [12])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [29]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

attributes = ['age', 'gender']

def MAP(_, row:NamedTuple):
    projection = tuple(getattr(row, attribute) for attribute in attributes)
    yield (projection, projection)
    
def REDUCE(proj:NamedTuple, _):
  yield (proj, proj)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]
    
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[((55, 'male'), (55, 'male')),
 ((25, 'female'), (25, 'female')),
 ((33, 'female'), (33, 'female'))]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [37]:
input_collection1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection2 = [
    User(id=0, age=55, gender='male', social_contacts=2000000),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, value):
    yield (value, value)
    
def REDUCE(id, values):
  yield (id, id)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection1] + [(u.id, u) for u in input_collection2]
    
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')),
 (User(id=0, age=55, social_contacts=2000000, gender='male'),
  User(id=0, age=55, social_contacts=2000000, gender='male'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [35]:
input_collection1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection2 = [
    User(id=0, age=55, gender='male', social_contacts=2000000),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, value):
    yield (value, value)
    
def REDUCE(id, values):
	if len(values) == 2:
		yield (id, id)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection1] + [(u.id, u) for u in input_collection2]
    
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [41]:
input_collection1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection2 = [
    User(id=0, age=55, gender='male', social_contacts=2000000),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, value, collection):
	if collection == '1':
		yield (value, 'R')
	else:
		yield (value, 'S')
    
def REDUCE(id, values):
	if all(i == 'R' for i in values):
		yield (id, id)
 
def RECORDREADER():
  return [(u.id, u, '1') for u in input_collection1] + [(u.id, u, '2') for u in input_collection2]
    
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [43]:
class Employee(NamedTuple):
  id: int
  salary: str

input_collection1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection2 = [
    Employee(id=0, salary=200),
    Employee(id=1, salary=300),
    Employee(id=2, salary=2000),
    Employee(id=3, salary=5555)
]

def MAP(_, value, collection):
	if collection == '1':
		yield (value.id, ('R', value.gender))
	else:
		yield (value.id, ('S', value.salary))
    
def REDUCE(id, values):
	S_list = [v for (t, v) in values if t == 'S']
	R_list = [v for (t, v) in values if t == 'R']
	for i in S_list:
		for j in R_list:
			yield (i, id, j)
	
 
def RECORDREADER():
  return [(u.id, u, '1') for u in input_collection1] + [(u.id, u, '2') for u in input_collection2]
    
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(200, 0, 'male'),
 (300, 1, 'female'),
 (2000, 2, 'female'),
 (5555, 3, 'female')]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [46]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

op = sum

def MAP(_, row:NamedTuple):
    yield (row.gender, row.social_contacts)
    
def REDUCE(id, values):
	yield (id, op(values))
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]
    
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('male', 20), ('female', 1540)]

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [1]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [7]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
	(j, k) = k1
	w = v1
	for i in range(I):
		yield ((i, k), (w * small_mat[i,j]))
  
def REDUCE(key, values):
  (i, k) = key
  yield (key, sum(values))

Проверьте своё решение

In [8]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [9]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [47]:
import numpy as np
import math
I = 2
J = 3
K = 4*10
M = np.random.rand(I,J)
N = np.random.rand(J,K)

def RECORDREADER():
	for j in range(J):
		# RECORDREADER не генерирует матрицы, а использует значения из M и N.
		# Это сделано, чтобы упростить проверку правильности решения.
		for k in range(K):
			yield ((j,k), N[j,k], 'N')
		for i in range(I):
			yield ((i,j), M[i,j], 'M')
      
def MAP(k1, v1, t):
	if t == 'N':
		(j, k) = k1
		w = v1
		for i in range(I):
			yield ((i, k), (j, w))
	else:
		(i, j) = k1
		v = v1
		for k in range(K):
			yield ((i, k), (j, v))
  
def REDUCE(key, values):
	(i, k) = key
	result = 0
	for j in range(J):
		result += math.prod(v for (jj, v) in values if jj == j)
	yield (key, result)
		  
# CHECK THE SOLUTION
reference_solution = np.matmul(M, N) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [48]:
import numpy as np
import math
import random

I = 2
J = 3
K = 4*10
M = np.random.rand(I,J)
N = np.random.rand(J,K)

maps = 2
reducers = 2

def RECORDREADER(ind1, ind2, mark):
	for i in range(ind1):
		for j in range(ind2):
			if mark == 'M':
				yield ((i,j), M[i,j], mark)
			else:
				yield ((i,j), N[i,j], mark)

def INPUTFORMAT():
  global maps
  yield RECORDREADER(I, J, 'M')
  yield RECORDREADER(J, K, 'N')      
      
def MAP(k1, v1, t):
	if t == 'N':
		(j, k) = k1
		w = v1
		for i in range(I):
			yield ((i, k), (j, w))
	else:
		(i, j) = k1
		v = v1
		for k in range(K):
			yield ((i, k), (j, v))
  
def REDUCE(key, values):
	(i, k) = key
	result = 0
	for j in range(J):
		result += math.prod(v for (jj, v) in values if jj == j)
	yield (key, result)

# CHECK THE SOLUTION
reference_solution = np.matmul(M, N) 

def asmatrix(reduce_output):
	reduce_output = list(reduce_output)
	mat = np.empty(shape=(I,K))
	for (partition, data) in reduce_output:
		for ((i,k), vw) in data:
			mat[i,k] = vw
	return mat

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
np.allclose(reference_solution, asmatrix(partitioned_output))

480 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [49]:
import numpy as np
import math
import random

I = 2
J = 3
K = 4*10
M = np.random.rand(I,J)
N = np.random.rand(J,K)

maps = 5
reducers = 3

def RECORDREADER(indices, mark):
	for (i, j) in indices:
		if mark == 'M':
			yield ((i,j), M[i,j], mark)
		else:
			yield ((i,j), N[i,j], mark)

def INPUTFORMAT():
	global maps
	M_maps = maps // 2
	N_maps = maps - M_maps
	M_split_size = I*J // M_maps
	N_split_size = J*K // N_maps

	M_indices = [(i, j) for i in range(I) for j in range(J)]
	np.random.shuffle(M_indices) # RECORDREADER генерируют случайное
	# подмножество элементов матрицы
	# и решение работает

	N_indices = [(j, k) for j in range(J) for k in range(K)]
	np.random.shuffle(N_indices)

	for i in range(0, I*J, M_split_size):
		yield RECORDREADER(M_indices[i:i+M_split_size], 'M')
	for i in range(0, J*K, N_split_size):
		yield RECORDREADER(N_indices[i:i+N_split_size], 'N')  
      
def MAP(k1, v1, t):
	if t == 'N':
		(j, k) = k1
		w = v1
		for i in range(I):
			yield ((i, k), (j, w))
	else:
		(i, j) = k1
		v = v1
		for k in range(K):
			yield ((i, k), (j, v))
  
def REDUCE(key, values):
	(i, k) = key
	result = 0
	for j in range(J):
		result += math.prod(v for (jj, v) in values if jj == j)
	yield (key, result)

# CHECK THE SOLUTION
reference_solution = np.matmul(M, N)
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

def asmatrix(reduce_output):
	reduce_output = list(reduce_output)
	mat = np.empty(shape=(I,K))
	for (partition, data) in reduce_output:
		for ((i,k), vw) in data:
			mat[i,k] = vw
	return mat

np.allclose(reference_solution, asmatrix(partitioned_output))

480 key-value pairs were sent over a network.


True